In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

def load_fashionmnist():
    # 学習データ
    x_train = np.load('../dataset/x_train.npy')
    x_train2 = x_train + np.random.rand(60000,28,28)
    x_train = np.concatenate([x_train,x_train2])
    y_train = np.load('../dataset/y_train.npy')
    y_train = np.concatenate([y_train,y_train])
    
    # テストデータ
    x_test = np.load('../dataset/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

import math
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

tf.reset_default_graph() # グラフのリセット

x_train, y_train, x_test = load_fashionmnist()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

#x = tf.placeholder(tf.float32, [None, 784])
#t = tf.placeholder(tf.float32, [None, 10])

def compute_l2_reg(params):
    l2_reg = 0
    for param in params:
        l2_reg += tf.reduce_sum(tf.square(param)) # 2 * tf.nn.l2_lossを使っても良い
    return l2_reg    

class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        self.W = tf.Variable(tf.random_uniform(shape=(in_dim, out_dim), minval=-0.08, maxval=0.08), name='W')
        self.b = tf.Variable(tf.zeros(out_dim), name='b')
        self.function = function
        
        self.params = [self.W, self.b]
    
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
class Dropout:
    def __init__(self, dropout_keep_prob=1.0):
        self.dropout_keep_prob = dropout_keep_prob
        self.params = []
    
    def __call__(self, x):
        # 訓練時のみdropoutを適用
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, keep_prob=self.dropout_keep_prob),
            false_fn=lambda: x
        )
def sgd(cost, params, eta=0.01):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        updates.append(param.assign_sub(eta * grad))
    return updates

# tf.log(0)によるnanを防ぐ
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
batch_size = 100 # バッチサイズ
n_epochs = 50 # epoch数

tf.reset_default_graph() # グラフのリセット

x = tf.placeholder(tf.float32, (None, 784)) # 入力データ
t = tf.placeholder(tf.float32, (None, 10)) # 教師データ
is_training = tf.placeholder(tf.bool) # 訓練時orテスト時

layers = [
    Dense(784, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 10, tf.nn.softmax)
]

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all

def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h

y = f_props(layers, x)
params_all = get_params(layers)
l2_reg = compute_l2_reg(params_all)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) + lmd * l2_reg

updates = sgd(cost, params_all, eta)
train = tf.group(*updates)

#n_epochs = 1
#batch_size = 100
n_batches = math.ceil(len(x_train) / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(n_epochs):
    x_train, y_train = shuffle(x_train, y_train)
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: y_train[start:end], is_training: True})
    y_pred, cost_valid_ = sess.run([y, cost], feed_dict={x: x_valid, t: y_valid, is_training: False})
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch + 1,
        cost_valid_,
        accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
y_pred = sess.run(y, feed_dict={x: x_test,is_training: True})
y_pred = np.argmax(y_pred, axis=1)
submission = pd.Series(y_pred, name='label')
submission.to_csv('../dataset/submission_pred.csv', header=True, index_label='id')

C:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


EPOCH: 1, Valid Cost: 1.260, Valid Accuracy: 0.665
EPOCH: 2, Valid Cost: 1.100, Valid Accuracy: 0.727
EPOCH: 3, Valid Cost: 1.011, Valid Accuracy: 0.765
EPOCH: 4, Valid Cost: 0.944, Valid Accuracy: 0.791
EPOCH: 5, Valid Cost: 0.899, Valid Accuracy: 0.800
EPOCH: 6, Valid Cost: 0.868, Valid Accuracy: 0.811
EPOCH: 7, Valid Cost: 0.831, Valid Accuracy: 0.814
EPOCH: 8, Valid Cost: 0.804, Valid Accuracy: 0.819
EPOCH: 9, Valid Cost: 0.785, Valid Accuracy: 0.820
EPOCH: 10, Valid Cost: 0.765, Valid Accuracy: 0.837
EPOCH: 11, Valid Cost: 0.743, Valid Accuracy: 0.849
EPOCH: 12, Valid Cost: 0.728, Valid Accuracy: 0.845
EPOCH: 13, Valid Cost: 0.711, Valid Accuracy: 0.850
EPOCH: 14, Valid Cost: 0.695, Valid Accuracy: 0.853
EPOCH: 15, Valid Cost: 0.680, Valid Accuracy: 0.848
EPOCH: 16, Valid Cost: 0.663, Valid Accuracy: 0.859
EPOCH: 17, Valid Cost: 0.653, Valid Accuracy: 0.857
EPOCH: 18, Valid Cost: 0.648, Valid Accuracy: 0.859
EPOCH: 19, Valid Cost: 0.638, Valid Accuracy: 0.862
EPOCH: 20, Valid Cost